In [8]:
import pandas as pd
from sql_con.conn import get_con_db

# sheet1

In [47]:
disease = pd.read_excel('./upload_data/常见疾病列表20220111.xlsx',sheet_name='【5大网站&宁夏】居民常见疾病列表291（重复≥4）')
# disease.drop(['Unnamed: 29'],axis=1,inplace=True)
disease = disease.dropna(axis=0, how='all')
disease = disease.dropna(axis=1, how='all')
disease['疾病'] = disease['疾病'].apply(lambda x: x.strip())
disease['in_superset'] = ''
disease

,序号,科室,学科,疾病,常见程度,模型上线,要求上线日期,典型症状,in_superset
0,1,呼吸内科,耳鼻咽喉头颈外科学,鼻炎,7,0,2022.01.18,"鼻炎,鼻炎,嗅觉下降,头痛,头昏,全身表现",
1,2,呼吸内科,耳鼻咽喉头颈外科学,扁桃体炎,7,0,2022.01.18,"发热,咽部不适,咽部疼痛,吞咽,呼吸困难,",
2,3,消化内科,消化病学,肠炎,7,0,2022.01.18,"腹痛,腹泻,便血,粘液便,厌食,口渴",
3,4,中医外科,骨科学,骨质增生,7,0,2022.01.18,"持续性隐痛,关节内有磨擦音,骨阻滞征,",
4,5,中医内科,耳鼻咽喉头颈外科学,慢性咽炎,7,0,2022.01.18,"咽部干燥,发痒,有异物感,分泌物不易咳出,干咳,恶心,干呕",
...,...,...,...,...,...,...,...,...,...
286,287,中医内科,耳鼻咽喉头颈外科学,中耳炎,4,0,2022.01.18,"耳痛,耳道流水或流脓,听力下降,耳鸣",
287,288,小儿科,神经病学,重症肌无力,4,0,2022.01.18,NaN,
288,289,神经内科,骨科学,椎动脉型颈椎病,4,0,2022.01.18,NaN,
289,290,妇科,妇产科学,子宫脱垂,4,0,2022.01.18,NaN,


In [48]:
con = get_con_db()
sql = '''
    select disease,apply_tag,old_remain from dw.golden_standard_superset
    where apply_tag = 1 and old_remain is null
'''
superset = pd.read_sql(sql,con)
superset['disease'] = superset['disease'].apply(lambda x: x.strip())
superset

,disease,apply_tag,old_remain
0,急性前列腺炎,1,None
1,先天性基底动脉畸形,1,None
2,先天性小脑动静脉畸形,1,None
3,大脑中动脉的蛛网膜下出血,1,None
4,ANCA相关性血管炎,1,None
...,...,...,...
1520,鼻唇囊肿,1,None
1521,鼻孢子菌病,1,None
1522,鼻疖,1,None
1523,鼻皮内痣,1,None


In [49]:
for i,row in disease.iterrows():
    disease_name = row['疾病']
    disease_name_len =  len(disease_name)
    for j,rows in superset.iterrows():
        superset_disease_name = rows['disease']
        count = 0
        for k in disease_name:
            if k in superset_disease_name:
                count += 1
            else:
                continue
        if count == disease_name_len:
            in_superset_value = disease.loc[disease['疾病'] == disease_name,'in_superset'].values
            if in_superset_value == '':
                disease.loc[disease['疾病'] == disease_name,'in_superset'] = superset_disease_name
            else:
                disease.loc[disease['疾病'] == disease_name,'in_superset'] += ','+ superset_disease_name

In [50]:
disease

,序号,科室,学科,疾病,常见程度,模型上线,要求上线日期,典型症状,in_superset
0,1,呼吸内科,耳鼻咽喉头颈外科学,鼻炎,7,0,2022.01.18,"鼻炎,鼻炎,嗅觉下降,头痛,头昏,全身表现",
1,2,呼吸内科,耳鼻咽喉头颈外科学,扁桃体炎,7,0,2022.01.18,"发热,咽部不适,咽部疼痛,吞咽,呼吸困难,",
2,3,消化内科,消化病学,肠炎,7,0,2022.01.18,"腹痛,腹泻,便血,粘液便,厌食,口渴","溃疡性结肠炎引起的关节病,乙状结肠炎,变形杆菌肠炎,嗜酸性粒细胞性胃肠炎,嗜酸细胞性胃肠炎,..."
3,4,中医外科,骨科学,骨质增生,7,0,2022.01.18,"持续性隐痛,关节内有磨擦音,骨阻滞征,",
4,5,中医内科,耳鼻咽喉头颈外科学,慢性咽炎,7,0,2022.01.18,"咽部干燥,发痒,有异物感,分泌物不易咳出,干咳,恶心,干呕",
...,...,...,...,...,...,...,...,...,...
286,287,中医内科,耳鼻咽喉头颈外科学,中耳炎,4,0,2022.01.18,"耳痛,耳道流水或流脓,听力下降,耳鸣",
287,288,小儿科,神经病学,重症肌无力,4,0,2022.01.18,NaN,"儿童型重症肌无力,先天性重症肌无力,妊娠合并重症肌无力,重症肌无力（中度全身型）,重症肌无力..."
288,289,神经内科,骨科学,椎动脉型颈椎病,4,0,2022.01.18,NaN,
289,290,妇科,妇产科学,子宫脱垂,4,0,2022.01.18,NaN,


In [51]:
disease['in_superset'].value_counts()

                                                                                               192
大脑动脉栓塞引起的脑梗死                                                                                     2
慢性胆石性胰腺炎                                                                                         2
慢性浅表性胃窦炎                                                                                         2
1型糖尿病性肾的并发症                                                                                      2
                                                                                              ... 
女性盆腔炎性疾病                                                                                         1
念珠菌性尿道口炎                                                                                         1
寻常痤疮,痘样痤疮,硬结性痤疮,粟粒坏死性痤疮,结节性痤疮                                                                    1
卵巢子宫内膜异位囊肿                                                                                       1
儿童型重症肌无力,先

In [52]:
disease.to_csv('./find_disease_sheet1.csv')

# sheet2

In [41]:
disease = pd.read_excel('./upload_data/常见疾病列表20220111.xlsx',sheet_name='【5大网站&宁夏】医生疾病列表1821')
disease.drop(['Unnamed: 29'],axis=1,inplace=True)
disease = disease.dropna(axis=0, how='all')
disease = disease.dropna(axis=1, how='all')
disease['疾病'] = disease['疾病'].apply(lambda x: x.strip())
disease['in_superset'] = ''
disease

,序号,学科,疾病,常见度,模型上线,要求上线日期,in_superset
0,1.0,呼吸病学,鼻炎,7.0,0.0,2022.01.31,
1,2.0,呼吸病学,扁桃体炎,7.0,0.0,2022.01.31,
2,3.0,消化病学,肠炎,7.0,0.0,2022.01.31,
3,4.0,中医,骨质增生,7.0,0.0,2022.01.31,
4,5.0,中医,慢性咽炎,7.0,0.0,2022.01.31,
...,...,...,...,...,...,...,...
1816,1817.0,妇科,子宫下垂,1.0,0.0,2022.01.31,
1817,1818.0,内分泌科,自身免疫性疾病,1.0,0.0,2022.01.31,
1818,1819.0,呼吸病学,纵隔气肿,1.0,0.0,2022.01.31,
1819,1820.0,妇科,足月小样儿,1.0,0.0,2022.01.31,


In [42]:
con = get_con_db()
sql = '''
    select disease,apply_tag,old_remain from dw.golden_standard_superset
    where apply_tag = 1 and old_remain is null
'''
superset = pd.read_sql(sql,con)
superset['disease'] = superset['disease'].apply(lambda x: x.strip())
superset

,disease,apply_tag,old_remain
0,急性前列腺炎,1,None
1,先天性基底动脉畸形,1,None
2,先天性小脑动静脉畸形,1,None
3,大脑中动脉的蛛网膜下出血,1,None
4,ANCA相关性血管炎,1,None
...,...,...,...
1520,鼻唇囊肿,1,None
1521,鼻孢子菌病,1,None
1522,鼻疖,1,None
1523,鼻皮内痣,1,None


In [43]:
for i,row in disease.iterrows():
    disease_name = row['疾病']
    disease_name_len =  len(disease_name)
    for j,rows in superset.iterrows():
        superset_disease_name = rows['disease']
        count = 0
        for k in disease_name:
            if k in superset_disease_name:
                count += 1
            else:
                continue
        if count == disease_name_len:
            in_superset_value = disease.loc[disease['疾病'] == disease_name,'in_superset'].values
            if in_superset_value == '':
                disease.loc[disease['疾病'] == disease_name,'in_superset'] = superset_disease_name
            else:
                disease.loc[disease['疾病'] == disease_name,'in_superset'] += ','+ superset_disease_name

In [44]:
disease

,序号,学科,疾病,常见度,模型上线,要求上线日期,in_superset
0,1.0,呼吸病学,鼻炎,7.0,0.0,2022.01.31,
1,2.0,呼吸病学,扁桃体炎,7.0,0.0,2022.01.31,
2,3.0,消化病学,肠炎,7.0,0.0,2022.01.31,"溃疡性结肠炎引起的关节病,乙状结肠炎,变形杆菌肠炎,嗜酸性粒细胞性胃肠炎,嗜酸细胞性胃肠炎,..."
3,4.0,中医,骨质增生,7.0,0.0,2022.01.31,
4,5.0,中医,慢性咽炎,7.0,0.0,2022.01.31,
...,...,...,...,...,...,...,...
1816,1817.0,妇科,子宫下垂,1.0,0.0,2022.01.31,
1817,1818.0,内分泌科,自身免疫性疾病,1.0,0.0,2022.01.31,
1818,1819.0,呼吸病学,纵隔气肿,1.0,0.0,2022.01.31,
1819,1820.0,妇科,足月小样儿,1.0,0.0,2022.01.31,


In [45]:
disease['in_superset'].value_counts()

                                                                                                                                                                                                                                                                                                         1399
初发性或维持性睡眠障碍[失眠症]                                                                                                                                                                                                                                                                                            4
大脑动脉栓塞引起的脑梗死                                                                                                                                                                                                                                                                                                3
胸椎腰椎椎间盘突出伴脊髓病                                                                                 

In [46]:
disease.to_csv('./find_disease_sheet2.csv')